In [1]:
import pandas as pd
import requests
import random
import json
import time
import glob
import os

In [25]:
tool2tag = {
    'Azure Machine Learning': {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench'},
    'Vertex AI': {'google-cloud-vertex-ai'},
    'Databricks': {'databricks-unity-catalog', 'databricks-ml'},
    'Amazon SageMaker': {'amazon-sagemaker', 'amazon-sagemaker-experiments'},
    'MLFlow': {'mlflow'},
    'DVC': {'dvc'},
    'Weights & Biases': {'wandb'},
    'ClearML': {'clearml'},
    'Comet': {'comet-ml'},
    'Neptune': {'neptune'},
    'Pachyderm': {'pachyderm'},
    'LakeFS': {'lakefs'}
}

In [10]:
path_dataset = '../Dataset'

path_so = os.path.join(path_dataset, 'Stack Overflow')
path_to = os.path.join(path_dataset, 'Tool-specific Others')
    
path_so_round1 = os.path.join(path_so, 'Round#1')
path_to_round1 = os.path.join(path_to, 'Round#1')
    
path_so_round1_raw = os.path.join(path_so_round1, 'Raw')
path_to_round1_raw = os.path.join(path_to_round1, 'Raw')
path_so_round1_filtered = os.path.join(path_so_round1, 'Filtered')

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

if not os.path.isdir(path_so):
    os.mkdir(path_so)
    
if not os.path.isdir(path_to):
    os.mkdir(path_to)
    
if not os.path.isdir(path_so_round1):
    os.mkdir(path_so_round1)
    
if not os.path.isdir(path_to_round1):
    os.mkdir(path_to_round1)
    
if not os.path.isdir(path_so_round1_raw):
    os.mkdir(path_so_round1_raw)
    
if not os.path.isdir(path_to_round1_raw):
    os.mkdir(path_to_round1_raw)
    
if not os.path.isdir(path_so_round1_filtered):
    os.mkdir(path_so_round1_filtered)

In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_round1_raw, 'Guild AI.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_round1_raw, 'Weights & Biases.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_round1_raw, 'SigOpt.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_round1_raw, 'DVC.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [4]:
tool_specific_post_summary = {}
for file_name in glob.glob(os.path.join(path_to_round1_raw, '*.json')):
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        tool_name = os.path.split(file_name)[1].split('.')[0]
        tool_specific_post_summary[tool_name] = len(json_data)
tool_specific_post_summary = pd.DataFrame(tool_specific_post_summary.items(), columns=['Tool', '#Posts'])
tool_specific_post_summary

,Tool,#Posts
0,Amazon SageMaker,528
1,Azure Machine Learning,1435
2,Databricks,154
3,Domino,13
4,DVC,315
5,Guild AI,115
6,MLFlow,280
7,Polyaxon,43
8,SigOpt,15
9,Vertex AI,297


In [17]:
df = pd.read_json(os.path.join(path_so_round1_raw, 'BigQuery.json'), lines=True)
df.head()

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_score,Question_tags,Question_view_count,Owner_creation_date,...,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score,Question_last_edit_date
0,Databricks display() function equivalent or al...,<p>I'm in the process of migrating current Dat...,5,3,2017-09-08 23:11:57.953 UTC,3.0,11,apache-spark|jupyter-notebook|databricks,23405,2016-06-15 18:26:00.217 UTC,...,3287,99,24,435,"<p>When you use Jupyter, instead of using df.s...",1.0,2020-03-18 15:19:00.54 UTC,2022-05-31 11:43:02.247 UTC,10.0,NaN
1,Spark: Read an inputStream instead of File,<p>I'm using SparkSQL in a Java application to...,1,0,2016-07-20 21:13:20.757 UTC,3.0,14,java|apache-spark|apache-spark-sql|spark-dataf...,10691,2014-11-22 00:35:18.107 UTC,...,3120,173,17,266,<p>You can use at least four different approac...,2.0,2016-07-25 20:08:43.747 UTC,NaN,4.0,NaN
2,databricks/spark/python/pyspark/serializers.py...,<p>When executing the following code provide b...,1,0,2022-06-03 21:18:43.163 UTC,NaN,1,python-3.x|pyspark|databricks|azure-databricks,255,2018-03-26 15:02:34.45 UTC,...,96,10,0,31,<p>The <code>SparkTrials</code> automatically ...,0.0,2022-06-09 15:32:38.88 UTC,NaN,1.0,NaN
3,Run Azure Databricks without Spark cluster,<p>I have used Domino Data Lab for a while and...,2,2,2018-10-11 17:46:49.543 UTC,0.0,6,python|azure|databricks,1829,2016-08-29 20:06:38.91 UTC,...,67,8,0,6,<p>You always have to have a &quot;cluster&quo...,0.0,2018-10-12 01:38:58.893 UTC,2020-12-17 09:14:27.683 UTC,8.0,NaN
4,What is the difference between sagemaker-pytor...,<p>When porting PyTorch code / models to SageM...,2,0,2022-09-12 20:03:07.273 UTC,NaN,0,amazon-web-services|pytorch|amazon-sagemaker,23,2022-09-08 15:07:33.073 UTC,...,3,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# create tag collection
tags = []
for key, value in tool2tag.items():
    tags.extend(value)

In [31]:
# split tags 
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [32]:
# count post number with different tags
arity = 0
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    print(f'{arity}+: {post_number}')
    arity = arity + 1

0+: 4997
1+: 213
2+: 16


In [69]:
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid.to_json(os.path.join(path_so_round1_filtered, 'valid_posts.json'),
                 indent=4, orient='records')

In [70]:
df_qualified = df_valid[df_valid['Question_score'] > -1]
df_qualified.to_json(os.path.join(
    path_so_round1_filtered, 'valid_posts_with_at_least_zero_score.json'), indent=4, orient='records')

In [71]:
df_qualified_accepted = df_qualified.dropna(subset=['Answer_body'])
df_qualified_accepted.to_json(os.path.join(
    path_so_round1_filtered, 'valid_posts_with_at_least_zero_score_and_answered_accepted.json'), indent=4, orient='records')

In [51]:
df_one_plus_tags = df[df['Question_valid_tags'].map(len) > 1]
df_one_plus_tags['Question_valid_tags'].map(str).unique().tolist()

["['azure-machine-learning-workbench', 'azure-machine-learning-studio']",
 "['azure-machine-learning-service', 'azure-machine-learning-studio']",
 "['azure-machine-learning-service', 'azure-machine-learning-workbench', 'azure-machine-learning-studio']",
 "['azure-machine-learning-service', 'mlflow']",
 "['mlflow', 'amazon-sagemaker']",
 "['azure-machine-learning-service', 'azure-machine-learning-workbench']",
 "['mlflow', 'azure-machine-learning-studio']",
 "['mlflow', 'dvc']",
 "['pachyderm', 'dvc']"]

In [66]:
# create a map from tag to tool
tag2tool = {}
for key, value in tool2tag.items():
    for elem in value:
        tag2tool.setdefault(elem, key)

In [72]:
df_qualified.assign(Question_exclusive_tag='')
posts_split = []

for index, row in df_qualified.iterrows():
    tool_list = []
    for tag in row['Question_valid_tags']:
        tool_list.append(tag2tool[tag])
    df_qualified.loc[index, 'Question_exclusive_tag'] = tool_list[0]
    if len(set(tool_list)) > 1:
        for tool in set(tool_list[1:]):
            series = row.copy()
            series['Question_exclusive_tag'] = tool
            posts_split.append(series)
df_qualified = df_qualified.drop(columns=['Question_valid_tags'])
df_qualified = pd.concat([df_qualified, pd.DataFrame(posts_split)], ignore_index=True)

C:\Users\zhimi\AppData\Local\Temp\ipykernel_13032\3894066259.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualified.loc[index, 'Question_exclusive_tag'] = tool_list[0]


In [ ]:
# Randomly sample a fraction of elements from a list
def random_sample(lst, frac=0.1, lowerbound=50, upperbound=100):
    sample_size = int(len(lst) * frac)
    if sample_size < lowerbound:
        sample_size = lowerbound
    if sample_size > min(len(lst), upperbound):
        sample_size = min(len(lst), upperbound)
    return random.sample(lst, sample_size)
random.seed(0)

In [ ]:
for file_name in glob.glob(os.path.join(raw_path, '*.json')):
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        sample_posts = random_sample(json_data)

    json_sample_posts = json.dumps(sample_posts, indent='\t')
    tool_name = os.path.split(file_name)[1].split('.')[0]
    with open(os.path.join(sample_others_round1_path, f'{tool_name}.json'), 'w') as outfile:
        outfile.write(json_sample_posts)

In [133]:
for name, group in df_exclusive_tag.groupby(['Question_valid_tags']):
    json_group = group.to_json(orient='records')
    parsed_group = json.loads(json_group)
    sample_group = random_sample(parsed_group)
    json_sample_group = json.dumps(sample_group, indent='\t')
    with open(os.path.join(sample_SO_round1_path, f'{name}.json'), 'w') as outfile:
        outfile.write(json_sample_group)